**DS530.V: Fairness and Interpretability in Data Science**

Class Project:  Fairness Analysis via Different ML Models on the UCI's Adult Dataset

*Author:* Utku Acar  
*Department:* Computer Science  
*Role:* Software Engineer / Researcher  

*My Secret Power:* I've got a knack for efficiently harnessing prompts and taming the wildest multi-dimensional data, especially in the realm of Images and Videos, through the magic of Deep Learning and Computer Vision.


*My Crime-Fighting Identity:* Get ready to meet the one and only Hyperion Solitude, because I'm about to dive into a world of adventure where data science meets supercharged strategies! 🚀🔍📊🦸‍♂️      

*Stay tuned for the thrilling odyssey, unfolding soon at https://github.com/hyperionsolitude. The adventure of a lifetime awaits! 🌟🎉*

Getting the dataset "Adult"

In [ ]:
!pip install ucimlrepo

## Task 1: Discuss why you need to treat fairness issues in this dataset

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)

# Accessing and printing the description from the metadata
for name, description in zip(adult.variables['name'], adult.variables['description']):
    print(name, ":", description)


## Task 2.1: Find a good black-box classification model to predict the target variable.

In [ ]:
import tensorflow as tf
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np

# Concatenate X and y for easier handling
data = pd.concat([X, y], axis=1)

data = data.replace('?', np.nan)

# Find rows with missing values and drop them
data = data.dropna()

binary_cols = ['sex','income']  # Add other binary column names if present
categorical_cols = [col for col in data.columns if col not in binary_cols and data[col].dtype == 'object']

# Separate binary and categorical columns
binary_data = data[binary_cols]
categorical_data = data[categorical_cols]

# Process categorical features into dummies
categorical_data = pd.get_dummies(categorical_data)

# Combine processed categorical features and unchanged binary features
processed_data = pd.concat([categorical_data, binary_data], axis=1)

# Separate X and y after processing
X = processed_data.drop('income', axis=1)
y = processed_data['income']

# Convert target variable to float32
y = (y == '>50K').astype('float32')

X['sex'] = X['sex'].map({'Female': 0, 'Male': 1})
X = X.astype('float32')

# Split the data into training, validation, and test sets (once)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Define ModelCheckpoint to save the best model
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=0)

# Initialize a list to store the test losses and accuracies
test_losses = []
test_accuracies = []

best_accuracy = 0.0  # Track the best accuracy
best_loss = float('inf')  # Track the best loss

# Run the model 10 times
for i in range(10):
    # Define the model
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Input layer
    model.add(Dense(64, activation='relu'))  # Hidden layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping, checkpoint], verbose=0)

    # Evaluate the model on the test data
    score = model.evaluate(X_test, y_test, verbose=0)
    print(f'Run {i+1}:')
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Append the test accuracy to the list
    test_losses.append(score[0])
    test_accuracies.append(score[1])

    # Check if the current accuracy is better than the previous best
    if score[1] > best_accuracy:
        print(f"New best accuracy found! Saving the model.")
        best_accuracy = score[1]
        model.save_weights('best_weights.keras')

    if score[0] < best_loss:
        best_loss = score[0]

    # Check for early stopping based on accuracy and loss improvement
    if i > 0 and (abs(test_accuracies[i] - best_accuracy) < 0.001 or abs(test_losses[i] - best_loss) < 0.001):
        print(f"No significant improvement from previous run. Stopping at iteration {i+1}.")
        break

## Task 2.2:  Calculate KPIs regarding fairness based on predictions of this black box model.

In [ ]:
!pip install aif360
!pip install fairlearn

In [ ]:
import pandas as pd
import numpy as np
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference, equalized_odds_ratio
from sklearn.metrics import confusion_matrix

def calculate_fairness_metrics(X_test, y_test, y_pred):
    y_pred_binary = (y_pred > 0.5).astype('int')

    eod_results = []
    insufficient_features = 0

    for feature in X_test.columns:
        group_0 = (X_test[feature] == 0)
        group_1 = (X_test[feature] == 1)

        if len(np.unique(y_test[group_0])) < 2 or len(np.unique(y_test[group_1])) < 2:
            insufficient_features += 1
            continue

        conf_matrix_0 = confusion_matrix(y_test[group_0], y_pred_binary[group_0])
        conf_matrix_1 = confusion_matrix(y_test[group_1], y_pred_binary[group_1])

        tn0, fp0, fn0, tp0 = conf_matrix_0.ravel()
        tn1, fp1, fn1, tp1 = conf_matrix_1.ravel()

        tpr0 = tp0 / (tp0 + fn0)
        fpr0 = fp0 / (fp0 + tn0)
        tpr1 = tp1 / (tp1 + fn1)
        fpr1 = fp1 / (fp1 + tn1)

        eod_value = equalized_odds_difference(y_test, y_pred_binary, sensitive_features=X_test[feature])
        pp_value = demographic_parity_difference(y_test, y_pred_binary, sensitive_features=X_test[feature])
        sp_value = equalized_odds_ratio(y_test, y_pred_binary, sensitive_features=X_test[feature])
        #mean_diff_nn = np.abs(np.mean(y_pred_binary[X_test[feature] == 1]) - np.mean(y_pred_binary[X_test[feature] == 0])) # Same with PP
        fpr_sensitive = np.mean(y_pred_binary[(y_test == 0) & (X_test[feature] == 1)])
        fpr_non_sensitive = np.mean(y_pred_binary[(y_test == 0) & (X_test[feature] == 0)])
        disparate_impact_nn = fpr_sensitive / fpr_non_sensitive

        eod_results.append((feature, eod_value, pp_value, sp_value, disparate_impact_nn, tpr0, fpr0, tpr1, fpr1))

    print("There are", insufficient_features, "features with insufficient data.")

    sorted_eod_results = sorted(eod_results, key=lambda x: x[1], reverse=True)

    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.precision', 3)

    kpi_df = pd.DataFrame(sorted_eod_results, columns=['Feature Name', 'EOD Value', 'Predictive Parity', 'Statistical Parity', 'Disparate Impact', 'TPR for 0', 'FPR for 0', 'TPR for 1', 'FPR for 1'])

    print("\n\n", kpi_df.to_string(index=False))

    return kpi_df

In [ ]:
import matplotlib.pyplot as plt

def create_table_image(dataframe, filename):
    formatted_df = dataframe.round(3)
    csv_filename = filename.replace('.png', '_data.csv')  # Adjust CSV filename
    formatted_df.to_csv(csv_filename, index=True)  # Save DataFrame to CSV file
    # Create the table figure
    plt.figure(figsize=(12, 8))
    plt.table(cellText=formatted_df.values,
              colLabels=formatted_df.columns,
              rowLabels=formatted_df.index,
              cellLoc='center',
              loc='center')
    plt.axis('off')

    # Save the table figure with the title below
    plt.savefig(filename, bbox_inches='tight', dpi=800)

In [ ]:
y_pred = model.predict(X_test)
kpi_nn_df = calculate_fairness_metrics(X_test, y_test,y_pred)
create_table_image(kpi_nn_df, 'kpi_metrics_table_nn.png')

## Task 3.1: Use a global surrogate model for a model agnostic explanation. (Logistic Regression)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create and train a logistic regression model on the predictions of the neural network
surrogate_model_LR = LogisticRegression(max_iter=1000)

# Obtain predictions of the neural network on the validation set
y_val_pred_nn = model.predict(X_train_val)

# Train the logistic regression model on the neural network predictions
surrogate_model_LR.fit(y_val_pred_nn, y_train_val)

# Get predictions of the neural network on the test set
y_test_pred_nn = model.predict(X_test)

# Use the trained logistic regression as a surrogate to predict on the neural network outputs
y_test_pred_surrogate_LR = surrogate_model_LR.predict(y_test_pred_nn)

# Evaluate the surrogate model's performance
surrogate_accuracy_LR = surrogate_model_LR.score(y_test_pred_nn, y_test)
print(f"Surrogate Model(Log-Reg) Accuracy: {surrogate_accuracy_LR}")

# Now, interpret the coefficients of the logistic regression model to understand feature importance
surrogate_coefficients_LR = surrogate_model_LR.coef_


## Task 3.2: Use a global surrogate model for a model agnostic explanation. (Decision Tree)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Get predictions of the neural network on the validation set
y_val_pred_nn = model.predict(X_train_val)

# Create a Decision Tree classifier as a surrogate model
surrogate_model_DT = DecisionTreeClassifier(max_depth=10)  # Adjust max_depth as needed

# Train the surrogate model on the neural network predictions and the original labels
surrogate_model_DT.fit(y_val_pred_nn, y_train_val)

# Get predictions of the neural network on the test set
y_test_pred_nn = model.predict(X_test)

# Get predictions of the surrogate model on the neural network predictions
y_test_pred_surrogate_DT = surrogate_model_DT.predict(y_test_pred_nn)

# Evaluate the surrogate model's performance (accuracy in this case)
surrogate_accuracy_DT = accuracy_score(y_test, y_test_pred_surrogate_DT)
print(f"Surrogate Model(Dec-Tre) Accuracy: {surrogate_accuracy_DT}")

# Optional: Visualize the surrogate tree (if depth is manageable)
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6),dpi=800)
plot_tree(surrogate_model_DT, filled=True, feature_names=X_train_val.columns)
plt.savefig('surrogate_model_DT.png')
plt.show()


## Task 4.1: Discuss if the fairness issues have increased or decreased in the global surrogate model using the KPIs introduced in the class.(Logistic Regression)

In [ ]:
def calculate_average_kpis(kpi_dataframe,modelname):
    # Replace 'inf' values with NaN
    kpi_dataframe.replace([np.inf, -np.inf], np.nan, inplace=True)
    avg_kpis = kpi_dataframe.iloc[:, 1:].mean()
    print(print("Average KPI Values for",modelname,"Model"))
    return avg_kpis

In [ ]:
def compare_kpi_features(model_names, *kpi_dataframes):
    # Transpose each dataframe
    transposed_dfs = [df.T for df in kpi_dataframes]

    # Concatenate the transposed dataframes along rows
    comparison_df = pd.concat(transposed_dfs, axis=1)

    # Set columns with model names
    comparison_df.columns = model_names

    # Calculate differences and percentage differences between models
    for idx, model_name in enumerate(model_names[1:], start=2):
        diff_col = f"Difference_{model_name}"
        perc_diff_col = f"Percentage_Difference_{model_name}"
        comparison_df[diff_col] = comparison_df[model_name] - comparison_df[model_names[0]]
        comparison_df[perc_diff_col] = (comparison_df[diff_col] / comparison_df[model_names[0]]) * 100

    return comparison_df

In [ ]:
kpi_lr_df = calculate_fairness_metrics(X_test, y_test,y_test_pred_surrogate_LR)
create_table_image(kpi_lr_df, 'kpi_metrics_table_lr.png')

In [ ]:
average_kpis_nn = calculate_average_kpis(kpi_nn_df,"Neural Network")
print(average_kpis_nn)

average_kpis_lr_sur = calculate_average_kpis(kpi_lr_df,"Logistic Regression Surrogate")
print(average_kpis_lr_sur)

comparison_nn_lr_sur = compare_kpi_features(["NN", "LR Sur"],average_kpis_nn, average_kpis_lr_sur)
print(comparison_nn_lr_sur)

create_table_image(comparison_nn_lr_sur, 'comparison_nn_lr_sur.png')

## Task 4.2: Discuss if the fairness issues have increased or decreased in the global surrogate model using the KPIs introduced in the class.(Decision Tree)

In [ ]:
kpi_dt_df = calculate_fairness_metrics(X_test, y_test,y_test_pred_surrogate_DT)
create_table_image(kpi_dt_df, 'kpi_metrics_table_dt.png')


In [ ]:
average_kpis_nn = calculate_average_kpis(kpi_nn_df,"Neural Network")
print(average_kpis_nn)

average_kpis_dt_sur = calculate_average_kpis(kpi_dt_df,"Decision Tree Surrogate")
print(average_kpis_dt_sur)

comparison_nn_dt_sur = compare_kpi_features(["NN", "DT Sur"],average_kpis_nn, average_kpis_dt_sur)
print(comparison_nn_dt_sur)

create_table_image(comparison_nn_dt_sur, 'comparison_nn_dt_sur.png')

## Task 5.1: Find a good interpretable classification model to predict the target variable. (Decision Tree)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from aif360.metrics import ClassificationMetric

# Train a Decision Tree classifier
interpretable_model_DT = DecisionTreeClassifier(max_depth=10)  # Adjust max_depth as needed
interpretable_model_DT.fit(X_train, y_train)

# Get predictions for test set
y_pred_interpretable_DT = interpretable_model_DT.predict(X_test)

# Calculate accuracy
accuracy_interpretable_DT = accuracy_score(y_test, y_pred_interpretable_DT)

plt.figure(figsize=(12, 6),dpi=800)
plot_tree(interpretable_model_DT, filled=True, feature_names=X_train.columns)
plt.savefig('interpretable_model_DT.png')
plt.show()

## Task 5.1: Find a good interpretable classification model to predict the target variable. (Logistic Regression)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from aif360.metrics import ClassificationMetric

# Train a Logistic Regression classifier
interpretable_model_LR = LogisticRegression(max_iter=1000)
interpretable_model_LR.fit(X_train, y_train)

# Get predictions for test set
y_pred_interpretable_LR = interpretable_model_LR.predict(X_test)

# Calculate accuracy

accuracy_interpretable_LR = accuracy_score(y_test, y_pred_interpretable_LR)


## Task 5.2.1: Compare the results of surrogate model and interpretable model in terms of model accuracy and fairness. (Logical Regression)

In [ ]:
kpi_lrmod_df = calculate_fairness_metrics(X_test, y_test,y_pred_interpretable_LR)
create_table_image(kpi_lrmod_df, 'kpi_metrics_table_lrmod.png')

In [ ]:
average_kpis_lr = calculate_average_kpis(kpi_lrmod_df,"Interpretable Logistic Regression")
print(average_kpis_lr)


comparison_surr_lr_int = compare_kpi_features(["LR Sur", "LR"],average_kpis_lr_sur, average_kpis_lr)
accuracy_row_LR = pd.Series({
    "LR Sur": surrogate_accuracy_LR,
    "LR": accuracy_interpretable_LR,
    "Difference_LR":  accuracy_interpretable_LR - surrogate_accuracy_LR,
    "Percentage_Difference_LR": 100 * (accuracy_interpretable_LR - surrogate_accuracy_LR) / accuracy_interpretable_LR
})
comparison_surr_lr_int = pd.concat([accuracy_row_LR.to_frame().T, comparison_surr_lr_int])
comparison_surr_lr_int = comparison_surr_lr_int.rename(index={comparison_surr_lr_int.index[0]: 'Accuracy'})
print(comparison_surr_lr_int)

create_table_image(comparison_surr_lr_int, 'comparison_surr_lr_int.png')

## Task 5.2.2: Compare the results of surrogate model and interpretable model in terms of model accuracy and fairness.(Decision Tree)

In [ ]:
kpi_dtmod_df = calculate_fairness_metrics(X_test, y_test,y_pred_interpretable_DT)
create_table_image(kpi_dtmod_df, 'kpi_metrics_table_dt.png')

In [ ]:
average_kpis_dt = calculate_average_kpis(kpi_dtmod_df,"Interpretable Decision Tree")
print(average_kpis_dt)

comparison_surr_dt_int = compare_kpi_features(["DT Sur", "DT"],average_kpis_dt_sur,average_kpis_dt)
accuracy_row_DT = pd.Series({
    "DT Sur": surrogate_accuracy_DT,
    "DT": accuracy_interpretable_DT,
    "Difference_DT": accuracy_interpretable_DT - surrogate_accuracy_DT,
    "Percentage_Difference_DT": 100 * (accuracy_interpretable_DT - surrogate_accuracy_DT) / accuracy_interpretable_DT
})

comparison_surr_dt_int = pd.concat([accuracy_row_DT.to_frame().T, comparison_surr_dt_int])
comparison_surr_dt_int = comparison_surr_dt_int.rename(index={comparison_surr_dt_int.index[0]: 'Accuracy'})
print(comparison_surr_dt_int)

create_table_image(comparison_surr_dt_int, 'comparison_surr_dt_int.png')

In [ ]:
import matplotlib.pyplot as plt

def create_table_image_sum(dataframe, filename):
    # Select the first and last 5 rows of the dataframe
    top_and_bottom = pd.concat([dataframe.head(5), dataframe.tail(5)])
    formatted_df = top_and_bottom.round(3)

    csv_filename = filename.replace('.png', '_data.csv')  # Adjust CSV filename
    formatted_df.to_csv(csv_filename, index=True)  # Save DataFrame to CSV file

    # Create the table figure with larger font size
    plt.figure(figsize=(12, 4))  # Adjust the height to make it more compact
    table = plt.table(cellText=formatted_df.values,
                      colLabels=formatted_df.columns,
                      rowLabels=formatted_df.index,
                      cellLoc='center',
                      loc='center')
    table.auto_set_font_size(True)
    # table.set_fontsize(12)  # Set your desired font size here
    table.scale(1.2, 1.2)   # Adjust the table size if needed
    plt.axis('off')
    # Adjust subplot parameters to remove excess space
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
    # Save the table figure with the title below
    plt.savefig(filename, bbox_inches='tight', dpi=800)


In [ ]:
create_table_image_sum(kpi_nn_df, 'kpi_metrics_table_nn_sum.png')

In [ ]:
create_table_image_sum(kpi_lr_df, 'kpi_metrics_table_lr_sum.png')

In [ ]:
create_table_image_sum(kpi_dt_df, 'kpi_metrics_table_dt_sum.png')

In [ ]:
create_table_image_sum(kpi_lrmod_df, 'kpi_metrics_table_lrmod_sum.png')

In [ ]:
create_table_image_sum(kpi_dtmod_df, 'kpi_metrics_table_dtmod_sum.png')

Extra code snippet for transforming csv's to Latex formatted tables

In [ ]:
import pandas as pd
import os

# Directory containing CSV files
directory = './'  # Replace './' with your directory containing CSV files

# Output file name
output_filename = 'output_table.tex'

# Function to convert CSV to LaTeX table
def csv_to_latex(csv_file):
    data = pd.read_csv(csv_file)
    data.columns = [header.replace('_', ' ') for header in data.columns]

    latex_table = "\\begin{table}[H]\n"
    latex_table += "  \\centering\n"
    latex_table += "  \\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}} " + "l" * len(data.columns) + "}\\toprule\n"
    latex_table += "    " + " & ".join(data.columns) + " \\\\ \\midrule\n"
    
    for _, row in data.iterrows():
        latex_table += "    " + " & ".join([str(val).replace('_', '\\_') for val in row.tolist()]) + " \\\\ \n"

    latex_table += "    \\bottomrule\n"
    latex_table += "  \\end{tabular*}\n"
    latex_table += f"  \\caption{{Comparison of KPIs: {os.path.splitext(os.path.basename(csv_file))[0].replace('_', ' ')}}}\n"
    latex_table += f"  \\label{{tab:{os.path.splitext(os.path.basename(csv_file))[0]}}}\n"
    latex_table += "\\end{table}\n\n"

    return latex_table

# Loop through CSV files in the directory and generate LaTeX tables
latex_output = ""
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        latex_output += csv_to_latex(file_path)

# Write the LaTeX tables to a single file
with open(output_filename, 'w') as file:
    file.write(latex_output)
